In [1]:
data ="""Once your text is split into tokens, you need to encode each token into a numerical
representation. You could potentially do this in a stateless way, such as by hashing each
token into a fixed binary vector, but in practice, the way you'd go about it is to build
an index of all terms found in the training data (the “vocabulary”), and assign a
unique integer to each entry in the vocabulary.
Something like this"""
data = data.replace("!", "")
data = data.replace("?", "")
data = data.lower().split()
print(data)

vocabulary = {}
for word in data:
    if word not in vocabulary:
        vocabulary[word] = len(vocabulary)

print(vocabulary)

['once', 'your', 'text', 'is', 'split', 'into', 'tokens,', 'you', 'need', 'to', 'encode', 'each', 'token', 'into', 'a', 'numerical', 'representation.', 'you', 'could', 'potentially', 'do', 'this', 'in', 'a', 'stateless', 'way,', 'such', 'as', 'by', 'hashing', 'each', 'token', 'into', 'a', 'fixed', 'binary', 'vector,', 'but', 'in', 'practice,', 'the', 'way', "you'd", 'go', 'about', 'it', 'is', 'to', 'build', 'an', 'index', 'of', 'all', 'terms', 'found', 'in', 'the', 'training', 'data', '(the', '“vocabulary”),', 'and', 'assign', 'a', 'unique', 'integer', 'to', 'each', 'entry', 'in', 'the', 'vocabulary.', 'something', 'like', 'this']
{'once': 0, 'your': 1, 'text': 2, 'is': 3, 'split': 4, 'into': 5, 'tokens,': 6, 'you': 7, 'need': 8, 'to': 9, 'encode': 10, 'each': 11, 'token': 12, 'a': 13, 'numerical': 14, 'representation.': 15, 'could': 16, 'potentially': 17, 'do': 18, 'this': 19, 'in': 20, 'stateless': 21, 'way,': 22, 'such': 23, 'as': 24, 'by': 25, 'hashing': 26, 'fixed': 27, 'binary': 

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


In [ ]:
!rm -r aclImdb/train/unsup

In [2]:
import numpy as np
def one_hot_encode_token(token):
    vector = np.zeros((len(vocabulary),))
    token_index = vocabulary[token]
    vector[token_index] = 1
    return vector


z = np.zeros((len(vocabulary),))
for key in vocabulary.keys():
    # z = np.hstack((z, one_hot_encode_token(key)))
    print(key)
print(z)


once
your
text
is
split
into
tokens,
you
need
to
encode
each
token
a
numerical
representation.
could
potentially
do
this
in
stateless
way,
such
as
by
hashing
fixed
binary
vector,
but
practice,
the
way
you'd
go
about
it
build
an
index
of
all
terms
found
training
data
(the
“vocabulary”),
and
assign
unique
integer
entry
vocabulary.
something
like
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
print(z)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
z.size

57

In [6]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(output_mode="int",)

import re
import string
import tensorflow as tf
def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(lowercase_string, f"[{re.escape(string.punctuation)}]", "")
def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)
text_vectorization = TextVectorization(output_mode="int",standardize=custom_standardization_fn, split=custom_split_fn,)

In [7]:
dataset = [
"I write, rewrite, and still rewrite again"
]
text_vectorization.adapt(dataset)

In [8]:
text_vectorization.get_vocabulary()

['', '[UNK]', 'rewrite', 'write', 'still', 'i', 'and', 'again']

In [9]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([5 3 2 6 4 2 7], shape=(7,), dtype=int64)


In [10]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and still rewrite again


In [11]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
"aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
"aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
"aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [12]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break


inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"This is about a mad scientist who creates a half shark - half man type critter on an uncharted island, then calls up all his old business and academic buddies to come and see his creation (evil laugh) but actually he wants his sharkman to kill them! Lots of bad GCI, goofy plot elements, and babes sweating in tight T-shirts follow.<br /><br />These monster movies all follow a similar formula, but this one spices things up with a bit of humor (I guarantee the folks who made this had tongues firmly planted in cheek), not to mention the sexy babes. But let's mention those sexy babes - several hot babes, in tight T-shirts, sweating profusely. One's in her undies at the beginning, another at the end. Thank you, thank you, makers of bad movies! The plot is full of goofy stuff, a guy drives up in a jeep, slams it right into a tree, then offers to fly everyone off the 

In [13]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
        ngrams=2,
        max_tokens=20000,
        output_mode="tf_idf",
)

In [14]:
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                    loss="binary_crossentropy",
                    metrics=["accuracy"])
    return model

In [16]:

tfidf_2gram_train_ds = train_ds.map(
                lambda x, y: (text_vectorization(x), y),
                num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
                lambda x, y: (text_vectorization(x), y),
                num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
                lambda x, y: (text_vectorization(x), y),
                num_parallel_calls=4)

In [17]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [18]:
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",save_best_only=True)
]

In [19]:
model.fit(
    tfidf_2gram_train_ds.cache(),
    validation_data=tfidf_2gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
625/625 [==============================] - 11s 16ms/step - loss: 0.4992 - accuracy: 0.7661 - val_loss: 0.3014 - val_accuracy: 0.8908
Epoch 2/10
625/625 [==============================] - 6s 9ms/step - loss: 0.3447 - accuracy: 0.8514 - val_loss: 0.5145 - val_accuracy: 0.7904
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.2967 - accuracy: 0.8763 - val_loss: 0.3739 - val_accuracy: 0.8656
Epoch 4/10
625/625 [==============================] - 9s 14ms/step - loss: 0.2719 - accuracy: 0.8907 - val_loss: 0.3578 - val_accuracy: 0.8824
Epoch 5/10
625/625 [==============================] - 9s 14ms/step - loss: 0.2508 - accuracy: 0.8985 - val_loss: 0.3490 - val_accuracy: 0.8864
Epoch 6/10
625/625 [==============================] - 8s 13ms/step - loss: 0.2453 - accuracy: 0.9016 - val_loss: 0.3411 - val_accuracy: 0.8862
Epoch 7/10
625/625 [==============================] - 8s 13ms/step - loss: 0.2337 - accuracy: 0.9040 - val_loss: 0.3479 - val_accuracy: 0.8876

In [20]:
model = keras.models.load_model("tfidf_2gram.keras")

print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 5s 6ms/step - loss: 0.2999 - accuracy: 0.8853
Test acc: 0.885
